# 1. Help Functions

In [159]:
################################################################################
### Python port of help_functions.R
### https://github.com/cran/hdm/blob/master/R/help_functions.R
################################################################################

################################################################################
### 1: Load modules
################################################################################

# Standard Python modules
import numpy as np
from sklearn.linear_model import LinearRegression as lm

################################################################################
### 2: Define functions
################################################################################

################################################################################
### 2.1: Functions which are not in the original R package
###      These are generally helper functions to allow an implementation which
###      reads as closely to the original R code as possible, and to ease a
###      Python implementation
################################################################################


# Define a function which turn a list or vector-like object into a proper two
# dimensional column vector
def cvec(a):
    """ Turn a list or vector-like object into a proper column vector

    Input
    a: List or vector-like object, has to be a potential input for np.array()

    Output
    vec: two dimensional NumPy array, with the first dimension weakly greater
         than the second (resulting in a column vector for a vector-like input)
    """
    # Conver input into a two dimensional NumPy array
    vec = np.array(a, ndmin=2)

    # Check whether the second dimension is strictly greater than the first
    # (remembering Python's zero indexing)
    if vec.shape[0] < vec.shape[1]:
        # If so, transpose the input vector
        vec = vec.T

    # Return the column vector
    return vec


# Define a function to mimic R's cor() function, which can take two matrices and
# return the correlation coefficients between the columns of the first and the
# columns of the second matrix
def cor(y, X):
    """ Return correlation coefficients between columns of matrices

    Inputs
    y: n by 1 NumPy array
    X: n by k NumPy array

    Outputs
    corr: list of length k, where the k-th element is the correlation
          coefficient between y and the k-th column of X
    """
    # Concatenate y and X into a single NumPy array
    yX = np.concatenate([y, X], axis=1)

    # Get the correlation coefficients between all columns of that array
    corr = np.corrcoef(yX, rowvar=False)

    # Get the first row, starting at the first off-diagonal element (these are
    # the correlation coefficients between y and each column of X
    corr = corr[0,1:]

    # Return the result
    return corr

################################################################################
### 2.2: Functions which are in the original R package
################################################################################


# Define a function which returns initial parameter guesses
def init_values(X, y, number=5, intercept=True):
    """ Return an initial parameter guess for a LASSO model

    Inputs
    y: n by 1 NumPy array, outcome variable
    X: n by k NumPy array, RHS variables

    Outputs
    residuals: n ny 1 NumPy array, residuals for initial parameter guess
    coefficients: k by 1 NumPy array, initial coefficient values
    """
    # Make sure y is a proper column vector
    y = cvec(y)

    # Get the absolute value of correlations between y and X
    corr = np.abs(cor(y, X))

    # Get the number of columns of X
    kx = X.shape[1]

    # Make an index selecting the five columns of X which are most correlated
    # with y (since .argsort() always sorts in increasing order, selecting from
    # the back gets the most highly correlated columns)
    index = corr.argsort()[-np.amin([number, kx]):]

    # Set up an array of coefficient guesses
    coefficients = np.zeros(shape=(kx, 1))

    # Regress y on the five most correlated columns of X, including an intercept
    # if desired
    reg = lm(fit_intercept=intercept).fit(X[:, index], y)

    # Replace the guesses for the estimated coefficients (note that .coef_ does
    # not return the estimated intercept, if one was included in the model)
    coefficients[index, :] = reg.coef_.T

    # Replace any NANs as zeros
    coefficients[np.isnan(coefficients)] = 0

    # Get the regression residuals
    residuals = y - reg.predict(X[:, index])

    # Return the residuals and coefficients
    return {'residuals': residuals, 'coefficients': coefficients}

# 2. LassoShooting 

In [160]:
################################################################################
### Python port of LassoShooting.fit.R
### https://github.com/cran/hdm/blob/master/R/LassoShooting.fit.R
################################################################################

################################################################################
### 1: Load modules
################################################################################

# Standard Python modules
import numpy as np

# Other parts of hdmpy
from hdmpy.help_functions import cvec, init_values

################################################################################
### 2: Define function
################################################################################

# Define shooting LASSO with variable dependent penalty terms
def LassoShooting_fit(x, y, lmbda, maxIter=1000, optTol=10**(-5),
                      zeroThreshold=10**(-6), XX=None, Xy=None,
                      beta_start=None):
    """ Shooting LASSO algorithm with variable dependent penalty weights

    Inputs
    x: n by p NumPy array, RHS variables
    y: n by 1 NumPy array, outcome variable
    lmbda: p by 1 NumPy array, variable dependent penalty terms. The j-th
           element is the penalty term for the j-th RHS variable.
    maxIter: integer, maximum number of shooting LASSO updated
    optTol: scalar, algorithm terminated once the sum of absolute differences
            between the updated and current weights is below optTol
    zeroThreshold: scalar, if any final weights are below zeroThreshold, they
                   will be set to zero instead
    XX: k by k NumPy array, pre-calculated version of x'x
    Xy: k by 1 NumPy array, pre-calculated version of x'y
    beta_start: k by 1 NumPy array, initial weights

    Outputs
    w: k by 1 NumPy array, final weights
    wp: k by m + 1 NumPy array, where m is the number of iterations the
        algorithm took. History of weight updates, starting with the initial
        weights.
    m: integer, number of iterations the algorithm took
    """
    # Make sure that y and lmbda are proper column vectors
    y = cvec(y)
    lmbda = cvec(lmbda)

    # Get number of observations n and number of variables p
    n, p = x.shape

    # Check whether XX and Xy were provided, calculate them if not
    if XX is None:
        XX = x.T @ x
    if Xy is None:
        Xy = x.T @ y

    # Check whether an initial value for the intercept was provided
    if beta_start is None:
        # If not, use init_values from help_functions, which will return
        # regression estimates for the five variables in x which are most
        # correlated with y, and initialize all other coefficients as zero
        beta = init_values(x, y, intercept=False)['coefficients']
    else:
        # Otherwise, use the provided initial weights
        beta = beta_start

    # Set up a history of weights over time, starting with the initial ones
    wp = beta

    # Keep track of the number of iterations
    m = 1

    # Create versions of XX and Xy which are just those matrices times two
    XX2 = XX * 2
    Xy2 = Xy * 2

    # Go through all iterations
    while m < maxIter:
        # Save the last set of weights (the .copy() is important, otherwise
        # beta_old will be updated every time beta is changed during the
        # following loop)
        beta_old = beta.copy()

        # Go through all parameters
        for j in np.arange(p):
            # Calculate the shoot
            S0 = XX2[j,:] @ beta - XX2[j,j] * beta[j,0] - Xy2[j,0]

            # Update the weights
            if np.isnan(S0).sum() >= 1:
                beta[j] = 0
            elif S0 > lmbda[j]:
                beta[j] = (lmbda[j] - S0) / XX2[j,j]
            elif S0 < -lmbda[j]:
                beta[j] = (-lmbda[j] - S0) / XX2[j,j]
            elif np.abs(S0) <= lmbda[j]:
                beta[j] = 0

        # Add the updated weights to the history of weights
        wp = np.concatenate([wp, beta], axis=1)

        # Check whether the weights are within tolerance
        if np.abs(beta - beta_old).sum() < optTol:
            # If so, break the while loop
            break

        # Increase the iteration counter
        m = m + 1

    # Set the final weights to the last updated weights
    w = beta

    # Set weights which are within zeroThreshold to zero
    w[np.abs(w) < zeroThreshold] = 0

    # Return the weights, history of weights, and iteration counter
    return {'coefficients': w, 'coef.list': wp, 'num.it': m}

In [161]:
################################################################################
### Python port of rlasso.R
### https://github.com/cran/hdm/blob/master/R/rlasso.R
################################################################################

################################################################################
### 1: Load modules
################################################################################

# Standard Python modules
import joblib as jbl
import multiprocess as mp
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression as lm

# Other parts of hdmpy
from hdmpy.help_functions import cvec, init_values
from hdmpy.LassoShooting_fit import LassoShooting_fit

################################################################################
### 2: Define functions
################################################################################

################################################################################
### 2.1: Functions which are not in the original R package
###      These are generally helper functions to allow an implementation which
###      reads as closely to the original R code as possible, and to ease a
###      Python implementation, including parallelizing the code
################################################################################


# Define a function to simulate distributions needed for calculating X-dependent
# penalty terms
def simul_pen(n, p, W, seed=0, fix_seed=True):
    # Check whether the seed needs to be fixed
    if fix_seed:
        # Simulate with provided seed
        g = norm.rvs(size=(n,1), random_state=seed) @ np.ones(shape=(1,p))
    else:
        # Simulate using whatever state the RNG is currently in
        g = norm.rvs(size=(n,1)) @ np.ones(shape=(1,p))

    # Calculate element of the distribution for the current draw of g
    s = n * np.amax(2 * np.abs(np.mean(W * g, axis=0)))

    # Return the result
    return s

################################################################################
### 2.2: Functions which are in the original R package
################################################################################


def lambdaCalculation(homoskedastic=False, X_dependent_lambda=False,
                      lambda_start=None, c=1.1, gamma=0.1, numSim=5000, y=None,
                      x=None, par=True, corecap=np.inf, fix_seed=True):
    # Get number of observations n and number of variables p
    n, p = x.shape

    # Get number of simulations to use (if simulations are necessary)
    R = numSim

    # Go through all possible combinations of homoskedasticy/heteroskedasticity
    # and X-dependent or independent error terms. The first two cases are
    # special cases: Handling the case there homoskedastic was set to None, and
    # where lambda_start was provided.
    #
    # 1) If homoskedastic was set to None (special case)
    if homoskedastic is None:
        # Initialize lambda
        lmbda0 = lambda_start

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        # Calculate the final vector of penalty terms
        lmbda = lmbda0 * Ups0

    # 2) If lambda_start was provided (special case)
    elif lambda_start is not None:
        # Check whether a homogeneous penalty term was provided (a scalar)
        if np.amax(cvec(lambda_start).shape) == 1:
            # If so, repeat that p times as the penalty term
            lmbda = np.ones(shape=(p,1)) * lambda_start
        else:
            # Otherwise, use the provided vector of penalty terms as is
            lmbda = lambda_start

    # 3) Homoskedastic and X-independent
    elif (homoskedastic == True) and (X_dependent_lambda == False):
        # Initilaize lambda
        lmbda0 = 2 * c * np.sqrt(n) * norm.ppf(1 - gamma/(2*p))

        # Use ddof=1 to be consistent with R's var() function
        Ups0 = np.sqrt(np.var(y, axis=0, ddof=1))

        # Calculate the final vector of penalty terms
        lmbda = np.zeros(shape=(p,1)) + lmbda0 * Ups0

    # 4) Homoskedastic and X-dependent
    elif (homoskedastic == True) and (X_dependent_lambda == True):
        psi = cvec((x**2).mean(axis=0))

        tXtpsi = (x.T / np.sqrt(psi)).T

        # Check whether to use parallel processing
        if par == True:
            # If so, get the number of cores to use
            cores = np.int(np.amin([mp.cpu_count(), corecap]))
        else:
            # Otherwise, use only one core (i.e. run sequentially)
            cores = 1

        # Get simulated distribution
        sim = jbl.Parallel(n_jobs=cores)(
            jbl.delayed(simul_pen)(
                n, p, tXtpsi, seed=l*20, fix_seed=fix_seed
            ) for l in np.arange(R)
        )

        # Convert it to a proper column vector
        sim = cvec(sim)

        # Initialize lambda based on the simulated quantiles
        lmbda0 = c * np.quantile(sim, q=1-gamma, axis=0)

        Ups0 = np.sqrt(np.var(y, axis=0, ddof=1))

        # Calculate the final vector of penalty terms
        lmbda = np.zeros(shape=(p,1)) + lmbda0 * Ups0

    # 5) Heteroskedastic and X-independent
    elif (homoskedastic == False) and (X_dependent_lambda == False):
        # The original includes the comment, "1=num endogenous variables"
        lmbda0 = 2 * c * np.sqrt(n) * norm.ppf(1 - gamma/(2*p*1))

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        lmbda = lmbda0 * Ups0

    # 6) Heteroskedastic and X-dependent
    elif (homoskedastic == False) and (X_dependent_lambda == True):
        eh = y

        ehat = eh @ np.ones(shape=(1,p))

        xehat = x * ehat

        psi = cvec((xehat**2).mean(axis=0)).T

        tXehattpsi = (xehat / ( np.ones(shape=(n,1)) @ np.sqrt(psi) ))

        # Check whether to use parallel processing
        if par == True:
            # If so, get the number of cores to use
            cores = np.int(np.amin([mp.cpu_count(), corecap]))
        else:
            # Otherwise, use only one core (i.e. run sequentially)
            cores = 1

        # Get simulated distribution
        sim = jbl.Parallel(n_jobs=cores)(
            jbl.delayed(simul_pen)(
                n, p, tXehattpsi, seed=l*20, fix_seed=fix_seed
            ) for l in np.arange(R)
        )

        # Convert it to a proper column vector
        sim = cvec(sim)

        # Initialize lambda based on the simulated quantiles
        lmbda0 = c * np.quantile(sim, q=1-gamma, axis=0)

        Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

        # Calculate the final vector of penalty terms
        lmbda = lmbda0 * Ups0

    # Return results
    return {'lambda0': lmbda0, 'lambda': lmbda, 'Ups0': Ups0}

In [162]:
import hdmpy
import pandas as pd
import numpy as np
import pyreadr
import math
import matplotlib.pyplot as plt
import random

# I downloaded the data that the author used
growth_read = pyreadr.read_r("../../data/GrowthData.RData")

# Extracting the data frame from rdata_read
growth = growth_read[ 'GrowthData' ]
#growth = growth.apply(pd.to_numeric)
#print(growth.dtypes)
growth['pop65'] = growth['pop65'].astype("int64")
#growth.astype('int64').dtypes

In [149]:
growth

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.031196,1,8.991064,0.0000,0.371898,0.014586,0.255,0.336,0.170,0.98,...,11.41,2.226,2.494,1.971,27.5,15.9,0.4407,0.4257,2.529,-0.011883
86,0.034096,1,8.025189,0.0050,0.296437,0.013615,0.108,0.117,0.093,1.00,...,1.95,0.510,0.694,0.362,20.2,15.7,0.1669,0.2201,25.553,-0.039080
87,0.046900,1,9.030137,0.0000,0.265778,0.008629,0.288,0.337,0.237,1.00,...,25.64,2.727,2.664,2.788,20.4,9.4,0.3238,0.3134,4.152,0.005175
88,0.039773,1,8.865312,0.0000,0.282939,0.005048,0.188,0.236,0.139,1.00,...,10.76,1.888,1.920,1.860,20.0,16.0,0.1845,0.1940,0.452,-0.029551


In [150]:
growth["pop65"].dtypes

dtype('int64')

In [151]:
# We create the main variables
y = growth['Outcome']
X = growth.drop('Outcome', 1)

# Create main variables
Y = growth['Outcome']
W = growth.drop(['Outcome','intercept', 'gdpsh465'], 1 )
D = growth['gdpsh465']

C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_33736/3518213529.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = growth.drop('Outcome', 1)
C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_33736/3518213529.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  W = growth.drop(['Outcome','intercept', 'gdpsh465'], 1 )


In [152]:
n, p = W.shape
gamma=0.1
c=1.1
x = W
y = Y


lmbda0 = 2 * c * np.sqrt(n) * norm.ppf(1 - gamma/(2*p*1))

Ups0 = (1 / np.sqrt(n)) * np.sqrt((y**2).T @ (x**2)).T

lmbda = lmbda0 * Ups0

In [153]:
x["pop65"]

0     12359
1      4630
2     19678
3      1482
4      3006
      ...  
85    13653
86     9093
87     8193
88    56226
89     3083
Name: pop65, Length: 90, dtype: int64

In [155]:
np.mean(lmbda)

4769.7559732148475

In [156]:
lmbda.sum()

286185.3583928907

In [158]:
lambdaCalculation(homoskedastic=False, X_dependent_lambda=False,
                      lambda_start=None, c=1.1, gamma=0.2, numSim=5000, y=Y,
                      x=W, par=True, corecap=np.inf, fix_seed=True)

{'lambda0': 61.26064367633269,
 'lambda': bmp1l            1.261436
 freeop           1.016037
 freetar          0.127779
 h65              0.495713
 hm65             0.623903
 hf65             0.377776
 p65              3.943929
 pm65             3.997176
 pf65             3.768832
 s65              1.937176
 sm65             2.113608
 sf65             1.851259
 fert65          20.104149
 mort65           0.311725
 lifee065        17.262366
 gpop1            0.097370
 fert1           21.085899
 mort1            0.373935
 invsh41          0.968080
 geetot1          0.153429
 geerec1          0.127346
 gde1             0.196848
 govwb1           0.522444
 govsh41          0.626674
 gvxdxe41         0.387331
 high65          21.602240
 highm65         28.684280
 highf65         16.269815
 highc65         11.911608
 highcm65        17.940494
 highcf65         7.030120
 human65         19.354445
 humanm65        21.930770
 humanf65        17.164104
 hyr65            0.660064
 hyrm65       

# 3. rlasso

In [175]:
################################################################################
### 3: Define classes
################################################################################

class rlasso:
    # Initialize gamma to None to get gamma=.1/log(n)
    def __init__(self, x, y, colnames=None, post=True, intercept=True,
                 model=True, homoskedastic=False, X_dependent_lambda=False,
                 lambda_start=None, c=1.1, gamma=None, numSim=5000, numIter=15,
                 tol=10**(-5), threshold=-np.inf, par=True, corecap=np.inf,
                 fix_seed=True):
        # Initialize internal variables
        if isinstance(x, pd.DataFrame) and colnames is None:
            colnames = x.columns

        self.x = np.array(x).astype(np.float32)
        self.y = cvec(y).astype(np.float32)

        self.n, self.p = self.x.shape

        if colnames is None:
            self.colnames = ['V' + str(i+1) for i in np.arange(self.p)]
        else:
            self.colnames = colnames

        # Unused line in the original code
        # ind_names = np.arange(self.p) + 1

        self.post = post
        self.intercept = intercept
        self.model = model
        self.homoskedastic = homoskedastic
        self.X_dependent_lambda = X_dependent_lambda
        self.lambda_start = lambda_start
        self.c = c

        if gamma is None:
            self.gamma = .1 / np.log(self.n)
        else:
            self.gamma = gamma

        self.numSim = numSim
        self.numIter = numIter
        self.tol = tol
        self.threshold = threshold

        self.par = par
        self.corecap = corecap
        self.fix_seed = fix_seed

        if (self.post == False) and (self.c is None):
            self.c = .5

        if (
                (self.post == False) and (self.homoskedastic == False)
                and (self.X_dependent_lambda == False)
                and (self.lambda_start == None) and (self.c == 1.1)
                and (self.gamma == .1 / np.log(self.n))
        ):
            self.c = .5

        # For now, instantiate estimate as None
        self.est = None

        # Calculate robust LASSO coefficients
        if self.intercept == True:
            meanx = cvec(self.x.mean(axis=0))

            self.x = self.x - np.ones(shape=(self.n,1)) @ meanx.T

            mu = self.y.mean()

            self.y = self.y - mu
        else:
            meanx = np.zeros(shape=(self.p,1))

            mu = 0

        normx = np.sqrt(np.var(self.x, axis=1, ddof=1))

        Psi = cvec(np.mean(self.x**2, axis=0))

        ind = np.zeros(shape=(self.p,1)).astype(bool)

        XX = self.x.T @ self.x

        Xy = self.x.T @ self.y

        startingval = init_values(self.x, self.y)['residuals']

        pen = lambdaCalculation(homoskedastic=self.homoskedastic,
                                X_dependent_lambda=self.X_dependent_lambda,
                                lambda_start=self.lambda_start, c=self.c,
                                gamma=self.gamma, numSim=self.numSim,
                                y=startingval, x=self.x, par=self.par,
                                corecap=self.corecap, fix_seed=self.fix_seed)

        lmbda = pen['lambda']
        Ups0 = Ups1 = pen['Ups0']
        lmbda0 = pen['lambda0']

        mm = 1
        s0 = np.sqrt(np.var(y, axis=0, ddof=1))

        while mm <= self.numIter:
            if (mm == 1) and self.post:
                coefTemp = (
                    LassoShooting_fit(self.x, self.y, lmbda/2, XX=XX,
                                      Xy=Xy)['coefficients']
                )
            else:
                coefTemp = (
                    LassoShooting_fit(self.x, self.y, lmbda, XX=XX,
                                      Xy=Xy)['coefficients']
                )

            coefTemp[np.isnan(coefTemp)] = 0

            ind1 = (np.abs(coefTemp) > 0)

            x1 = self.x[:, ind1[:,0]]

            if x1.shape[1] == 0:
                if self.intercept:
                    intercept_value = np.mean(self.y + mu)

                    coef = np.zeros(shape=(self.p+1,1))

                    coef = (
                        pd.DataFrame(coef,
                                     index=['(Intercept)']+list(self.colnames))
                    )
                else:
                    intercept_value = np.mean(self.y)

                    coef = np.zeros(shape=(self.p,1))

                    coef = pd.DataFrame(coef, index=self.colnames)

                self.est = {
                    'coefficients': coef,
                    'beta': np.zeros(shape=(self.p,1)),
                    'intercept': intercept_value,
                    'index': pd.DataFrame(np.zeros(shape=(self.p,1)).astype(
                        bool),
                                          index=self.colnames),
                    'lambda': lmbda,
                    'lambda0': lmbda0,
                    'loadings': Ups0,
                    'residuals': self.y - np.mean(self.y),
                    'sigma': np.var(self.y, axis=0, ddof=1),
                    'iter': mm,
                    #'call': Not a Python option
                    'options': {'post': self.post, 'intercept': self.intercept,
                                'ind.scale': ind, 'mu': mu, 'meanx': meanx}
                }

                if self.model:
                    self.est['model'] = self.x
                else:
                    self.est['model'] = None

                self.est['tss'] = self.est['rss'] = (
                    ((self.y - np.mean(self.y))**2).sum()
                )

                self.est['dev']: self.y - np.mean(self.y)
                # In R, return() breaks while loops
                return

            # Refinement variance estimation
            if self.post:
                reg = lm(fit_intercept=False).fit(x1, self.y)

                coefT = reg.coef_.T

                coefT[np.isnan(coefT)] = 0

                e1 = self.y - x1 @ coefT

                coefTemp[ind1[:,0]] = coefT
            else:
                e1 = self.y - x1@ coefTemp[ind1[:,0]]

            s1 = np.sqrt(np.var(e1, ddof=1))

            # Homoskedastic and X-independent
            if (
                    (self.homoskedastic == True)
                    and (self.X_dependent_lambda == False)
            ):
                Ups1 = s1 * Psi

                lmbda = pen['lambda0'] * Ups1

            # Homoskedastic and X-dependent
            elif (
                    (self.homoskedastic == True)
                    and (self.X_dependent_lambda == True)
            ):
                Ups1 = s1 * Psi

                lmbda = pen['lambda0'] * Ups1

            # Heteroskedastic and X-independent
            elif (
                    (self.homoskedastic == False)
                    and (self.X_dependent_lambda == False)
            ):
                Ups1 = (
                    (1/np.sqrt(self.n)) * np.sqrt((e1**2).T @ self.x**2).T
                )

                lmbda = pen['lambda0'] * Ups1

            # Heteroskedastic and X-dependent
            elif (
                    (self.homoskedastic == False)
                    and (self.X_dependent_lambda == True)
            ):
                lc = lambdaCalculation(homoskedastic=self.homoskedastic,
                                       X_dependent_lambda=
                                       self.X_dependent_lambda,
                                       lambda_start=self.lambda_start,
                                       c=self.c, gamma=self.gamma,
                                       numSim=self.numSim, y=e1, x=self.x,
                                       par=self.par, corecap=self.corecap,
                                       fix_seed=self.fix_seed)

                Ups1 = lc['Ups0']

                lmbda = lc['lambda']

            # If homoskedastic is set to None
            elif self.homoskedastic is None:
                Ups1 = (
                    (1/np.sqrt(self.n)) * np.sqrt((e1**2).T @ self.x**2).T
                )

                lmbda = pen['lambda0'] * Ups1

            mm = mm + 1

            if np.abs(s0 - s1) < self.tol:
                break

            s0 = s1

        if x1.shape[1] == 0:
            #coefTemp = None
            ind1 = np.zeros(shape=(self.p,1))

        coefTemp = cvec(coefTemp)

        coefTemp[np.abs(coefTemp) < self.threshold] = 0

        coefTemp = pd.DataFrame(coefTemp, index=self.colnames)

        ind1 = cvec(ind1)

        ind1 = pd.DataFrame(ind1, index=self.colnames)

        if self.intercept:
            if mu is None:
                mu = 0
            if meanx is None:
                meanx = np.zeros(shape=(coefTemp.shape[0],1))
            if ind.sum() == 0:
                intercept_value = mu - (meanx * coefTemp).sum()
            else:
                intercept_value = mu - (meanx * coefTemp).sum()
        else:
            intercept_value = np.nan

        if self.intercept:
            beta = (
                np.concatenate([cvec(intercept_value), coefTemp.values], axis=0)
            )

            beta = pd.DataFrame(beta, index=['(Intercept)']+list(self.colnames))
        else:
            beta = coefTemp

        s1 = np.sqrt(np.var(e1, ddof=1))

        self.est = {
            'coefficients': beta,
            'beta': pd.DataFrame(coefTemp, index=self.colnames),
            'intercept': intercept_value,
            'index': ind1,
            'lambda': pd.DataFrame(lmbda, index=self.colnames),
            'lambda0': lmbda0,
            'loadings': Ups1,
            'residuals': cvec(e1),
            'sigma': s1,
            'iter': mm,
            #'call': Not a Python option
            'options': {'post': self.post, 'intercept': self.intercept,
                        'ind.scale': ind, 'mu': mu, 'meanx': meanx},
            'model': model
        }

        if model:
            self.x = self.x + np.ones(shape=(self.n,1)) @ meanx.T

            self.est['model'] = self.x
        else:
            self.est['model'] = None

        self.est['tss'] = ((self.y - np.mean(self.y))**2).sum()
        self.est['rss'] = (self.est['residuals']**2).sum()
        self.est['dev'] = self.y - np.mean(self.y)
        self.est['Xy'] = Xy        
        self.est['startingval'] = startingval
        self.est['pen'] = pen
        self.est['ind1'] = ind1 
        
        self.est['x1'] = x1        
        
        

In [180]:
pd.set_option('display.max_rows', None)


In [182]:
rlasso( W, Y, post=True ).est['residuals']

array([[-0.06099874],
       [ 0.08877641],
       [ 0.00895057],
       [ 0.02107867],
       [-0.01702298],
       [-0.01169348],
       [ 0.00923142],
       [-0.01614626],
       [-0.02454968],
       [-0.00774048],
       [ 0.02026233],
       [ 0.08610442],
       [-0.04979783],
       [ 0.03295552],
       [ 0.06865422],
       [ 0.00147667],
       [-0.01999469],
       [-0.02679093],
       [-0.09468942],
       [-0.00710572],
       [ 0.03746511],
       [ 0.07588129],
       [ 0.15053784],
       [ 0.05294329],
       [-0.00612673],
       [ 0.02626686],
       [ 0.05945478],
       [ 0.0360154 ],
       [ 0.02527513],
       [ 0.01813254],
       [ 0.02592292],
       [-0.00516014],
       [ 0.05941767],
       [ 0.0097508 ],
       [ 0.01580399],
       [ 0.01596973],
       [ 0.00773677],
       [ 0.03675017],
       [-0.00539555],
       [-0.01092044],
       [-0.01907675],
       [-0.00973178],
       [ 0.00194517],
       [ 0.05472925],
       [-0.09396082],
       [-0

In [ ]:
rlasso.default <- function(x, y, post = TRUE, intercept = TRUE, model = TRUE, 
                           penalty = list(homoscedastic = FALSE, X.dependent.lambda = FALSE, lambda.start = NULL, c = 1.1, gamma = .1/log(n)),
                           control = list(numIter = 15, tol = 10^-5, threshold = NULL),...) {
  x <- as.matrix(x)
  y <- as.matrix(y)
  
  n <- dim(x)[1]
  p <- dim(x)[2]
  
  if (is.null(colnames(x)))
    colnames(x) <- paste("V", 1:p, sep = "")
  ind.names <- 1:p
  # set options to default values if missing
  if (!exists("homoscedastic", where = penalty))  penalty$homoscedastic = "FALSE"
  if (!exists("X.dependent.lambda", where = penalty))  penalty$X.dependent.lambda = "FALSE"
  if (!exists("gamma", where = penalty))  penalty$gamma = 0.1/log(n)
  
  if (penalty$homoscedastic=="none" & !exists("lambda.start", where=penalty)) stop("lambda.start must be provided!")
  # checking input numIter, tol
  if (!exists("numIter", where = control)) {
    control$numIter = 15
  }
  
  if (!exists("tol", where = control)) {
    control$tol = 10^-5
  }
  
  #if (post==FALSE & (!exists("c", where = penalty) | is.na(match("penalty", names(as.list(match.call)))))) {
  if (post==FALSE & (!exists("c", where = penalty))) {  
    penalty$c = 0.5
  }
  
  default_pen <-  list(homoscedastic = FALSE, X.dependent.lambda = FALSE, lambda.start = NULL, c = 1.1, gamma = .1/log(n))
  if (post==FALSE &  isTRUE(all.equal(penalty, default_pen))) {  
    penalty$c = 0.5
  }
  
  # Intercept handling and scaling
  if (intercept) {
    meanx <- colMeans(x)
    x <- scale(x, meanx, FALSE)
    mu <- mean(y)
    y <- y - mu
  } else {
    meanx <- rep(0, p)
    mu <- 0
  }
  
  normx <- sqrt(apply(x, 2, var))
  Psi <- apply(x, 2, function(x) mean(x^2)) 
  ind <- rep(FALSE, p) #
  
  # variables with low variation are taken out, because normalization is not reliable
  # eps <- 10^-9  # precision for scaling
  #ind <- which(normx < eps)
  #if (length(ind) != 0) {
  #  x <- x[, -ind]
  #  normx <- normx[-ind]
  #  ind.names <- ind.names[-ind]
  #  p <- dim(x)[2]
  #  if (!is.null(penalty$lambda.start)) {
  #    penalty$lambda.start <- penalty$lambda.start[-ind]
  #  }
  #}
  
  #
  
  XX <- crossprod(x)
  Xy <- crossprod(x, y)
  
  startingval <- init_values(x,y)$residuals
  pen <- lambdaCalculation(penalty = penalty, y = startingval, x = x)
  lambda <- pen$lambda
  Ups0 <- Ups1 <- pen$Ups0
  lambda0 <- pen$lambda0
  
  mm <- 1
  s0 <- sqrt(var(y))
  
  while (mm <= control$numIter) {
    # calculation parameters
    #coefTemp <- LassoShooting.fit(x, y, lambda, XX = XX, Xy = Xy)$coefficients
    #xn <- t(t(x)/as.vector(Ups1))
    if (mm==1 && post) {
      coefTemp <- LassoShooting.fit(x, y, lambda/2, XX = XX, Xy = Xy)$coefficients
      #lasso.reg <- glmnet::glmnet(xn, y, family = c("gaussian"), alpha = 1,
      #                            lambda = lambda0/(2*n)/2, standardize = FALSE, intercept = FALSE)
      #lasso.reg <- glmnet::glmnet(x, y, family = c("gaussian"), alpha = 1,
      #                           lambda = lambda0/(2*n)/2, standardize = FALSE, intercept = FALSE,  penalty.factor = Ups1)
    } else {
      coefTemp <- LassoShooting.fit(x, y, lambda, XX = XX, Xy = Xy)$coefficients
      #lasso.reg <- glmnet::glmnet(xn, y, family = c("gaussian"), alpha = 1,
      #                           lambda = lambda0/(2*n), standardize = FALSE, intercept = FALSE)
      #lasso.reg <- glmnet::glmnet(x, y, family = c("gaussian"), alpha = 1,
      #                           lambda = lambda0/(2*n), standardize = FALSE, intercept = FALSE,  penalty.factor = Ups1)
    }
    #coefTemp <- as.vector(lasso.reg$beta)
    #names(coefTemp) <- colnames(x)
    coefTemp[is.na(coefTemp)] <- 0
    ind1 <- (abs(coefTemp) > 0)
    x1 <- as.matrix(x[, ind1, drop = FALSE])
    if (dim(x1)[2] == 0) {
      if (intercept) {
        intercept.value <- mean(y + mu)
        coef <- rep(0,p+1)
        names(coef) <- c("intercept", colnames(x)) #c("intercept", names(coefTemp))
      } else {
        intercept.value <- mean(y)
        coef <- rep(0,p)
        names(coef) <- colnames(x) #names(coefTemp)
      }
      est <- list(coefficients = coef, beta=rep(0,p), intercept=intercept.value, index = rep(FALSE, p),
                  lambda = lambda, lambda0 = lambda0, loadings = Ups0, residuals = y -
                    mean(y), sigma = var(y), iter = mm, call = match.call(),
                  options = list(post = post, intercept = intercept, ind.scale=ind, 
                                 control = control, mu = mu, meanx = meanx))
      if (model) {
        est$model <- x
      } else {
        est$model <- NULL
      }
      est$tss <- est$rss <- sum((y - mean(y))^2)
      est$dev <- y - mean(y)
      class(est) <- "rlasso"
      return(est)
    }
    
    # refinement variance estimation
    if (post) {
      reg <- lm(y ~ -1 + x1)
      coefT <- coef(reg)
      coefT[is.na(coefT)] <- 0
      e1 <- y - x1 %*% coefT
      coefTemp[ind1] <- coefT
    }
    if (!post) {
      e1 <- y - x1 %*% coefTemp[ind1]
    }
    s1 <- sqrt(var(e1))
    
    # homoscedatic and X-independent
    if (penalty$homoscedastic == TRUE && penalty$X.dependent.lambda == FALSE) {
      Ups1 <- c(s1)*Psi
      #lambda <- rep(pen$lambda0 * s1, p)
      lambda <- pen$lambda0*Ups1
    }
    # homoscedatic and X-dependent
    if (penalty$homoscedastic == TRUE && penalty$X.dependent.lambda == TRUE) {
      Ups1 <- c(s1)*Psi
      #lambda <- rep(pen$lambda0 * s1, p)
      lambda <- pen$lambda0 * Ups1
    }
    # heteroscedastic and X-independent
    if (penalty$homoscedastic == FALSE && penalty$X.dependent.lambda == FALSE) {
      Ups1 <- 1/sqrt(n) * sqrt(t(t(e1^2) %*% (x^2)))
      lambda <- pen$lambda0 * Ups1
    }
    
    # heteroscedastic and X-dependent
    if (penalty$homoscedastic == FALSE && penalty$X.dependent.lambda == TRUE) {
      lc <- lambdaCalculation(penalty, y=e1, x=x)
      Ups1 <- lc$Ups0
      lambda <- lc$lambda
    }
    
    
    
    # none
    if (penalty$homoscedastic == "none") {
      if (is.null(penalty$lambda.start)) stop("Argument lambda.start required!")
      Ups1 <- 1/sqrt(n) * sqrt(t(t(e1^2) %*% (x^2)))
      lambda <- pen$lambda0 * Ups1
    }
    
    mm <- mm + 1
    if (abs(s0 - s1) < control$tol) {
      break
    }
    s0 <- s1
  }
  
  if (dim(x1)[2] == 0) {
    coefTemp = NULL
    ind1 <- rep(0, p)
  }
  coefTemp <- as.vector(coefTemp)
  coefTemp[abs(coefTemp) < control$threshold] <- 0
  ind1 <- as.vector(ind1)
  coefTemp <- as.vector(as.vector(coefTemp))
  names(coefTemp) <- names(ind1) <- colnames(x)
  if (intercept) {
    if (is.null(mu)) mu <-0
    if (is.null(meanx))  meanx <-  rep(0, length(coefTemp))  #<- 0
    if (sum(ind)==0) {
      intercept.value <- mu - sum(meanx*coefTemp)
    } else {
      intercept.value <- mu - sum(meanx*coefTemp) #sum(meanx[-ind]*coefTemp)
    }
  } else {
    intercept.value <- NA
  }
  
  #if (intercept) {
  #  e1 <- y - x1 %*% coefTemp[ind1] - intercept.value 
  #} else {
  #  e1 <- y - x1 %*% coefTemp[ind1]
  #}
  if (intercept) {
    beta <- c(intercept.value, coefTemp)
    names(beta)[1] <- "(Intercept)"
  } else {
    beta <- coefTemp
  }
  
  s1 <- sqrt(var(e1))
  est <- list(coefficients = beta, beta=coefTemp, intercept=intercept.value, index = ind1, lambda = lambda,
              lambda0 = lambda0, loadings = Ups1, residuals = as.vector(e1), sigma = s1,
              iter = mm, call = match.call(), options = list(post = post, intercept = intercept,
                                                             control = control, penalty = penalty, ind.scale=ind,
                                                             mu = mu, meanx = meanx), model=model)
  if (model) {
    x <- scale(x, -meanx, FALSE)
    est$model <- x
  } else {
    est$model <- NULL
  }
  est$tss <- sum((y - mean(y))^2)
  est$rss <- sum(est$residuals^2)
  est$dev <- y - mean(y)
  class(est) <- "rlasso"
  return(est)
}